In [2]:
# importing the required modules
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
 
# specifying the path to csv files
income_1csv = Path("./Resources/Income/Australian Bureau of Statistics.csv")
income_2csv = Path("./Resources/Income/Australian Bureau of Statistics (1).csv")
income_3csv = Path("./Resources/Income/Australian Bureau of Statistics (2).csv")
income_4csv = Path("./Resources/Income/Australian Bureau of Statistics (3).csv")

income_1 = pd.read_csv(income_1csv)
income_2 = pd.read_csv(income_2csv)
income_3 = pd.read_csv(income_3csv)
income_4 = pd.read_csv(income_4csv)

#income_1.head()
new_header = income_2.iloc[0] #grab the first row for the header
income_2 = income_2[1:] #take the data less the header row
income_2.columns = new_header #set the header row as the df header
income_2.columns = income_2.columns.fillna('Unnamed: 0')

In [3]:
income_1.loc[2:2]

,Unnamed: 0,August 2021,August 2020,August 2015,2020 to 2021(% change),2015 to 2021(% change p.a.)
2,Total,$1200,$1150,$1000,4.3%,3.3%


In [4]:
income_2_sub_df = income_2.loc[4:4][['Unnamed: 0','August 2023','August 2018']]
income_2_sub_df.columns = ['Unnamed: 0', '2023','2018']
income_2_sub_df

,Unnamed: 0,2023,2018
4,Total,"$1,300","$1,075"


In [5]:
income_3_sub_df = income_3.loc[2:2][['Unnamed: 0', 'August 2022', 'August 2021']]
income_3_sub_df

,Unnamed: 0,August 2022,August 2021
2,Total,"$1,250","$1,200"


In [6]:
income_4_sub_df = income_4.loc[2:2][['Unnamed: 0', 'August 2020', 'August 2019']]
income_4_sub_df

,Unnamed: 0,August 2020,August 2019
2,Total,$1150,$1100


In [7]:
concat_3_4_df = pd.concat([income_3_sub_df, income_4_sub_df], axis = 1)#.columns
concat_3_4_df.columns = ['Unnamed: 0', '2022','2021','to_be_removed','2020','2019']

concat_3_4_df = concat_3_4_df.drop(concat_3_4_df.columns[3], axis=1)
concat_3_4_df

,Unnamed: 0,2022,2021,2020,2019
2,Total,"$1,250","$1,200",$1150,$1100


In [27]:
merged_df = pd.merge(income_2_sub_df, concat_3_4_df, on = 'Unnamed: 0')
merged_df = merged_df.transpose()
new_header = merged_df.iloc[0] #grab the first row for the header
merged_df = merged_df[1:] #take the data less the header row
merged_df.columns = new_header #set the header row as the df header
merged_df.reset_index(inplace = True)
merged_df.columns = ["Year", "Income Per Week"]
merged_df

,Year,Income Per Week
0,2023,"$1,300"
1,2018,"$1,075"
2,2022,"$1,250"
3,2021,"$1,200"
4,2020,$1150
5,2019,$1100


In [28]:
merged_df['Income Per Week'] = merged_df['Income Per Week'].str.replace("$","",regex=False)
merged_df['Income Per Week'] = merged_df['Income Per Week'].str.replace(",","",regex=False)
merged_df['Income Per Week'] = merged_df['Income Per Week'].astype(int)
merged_df['Year'] = merged_df['Year'].astype(int)
merged_df = merged_df.sort_values('Year')
merged_df

,Year,Income Per Week
1,2018,1075
5,2019,1100
4,2020,1150
3,2021,1200
2,2022,1250
0,2023,1300


In [30]:
merged_df['Income Per Annum'] = merged_df['Income Per Week'] * 52
merged_df

,Year,Income Per Week,Income Per Annum
1,2018,1075,55900
5,2019,1100,57200
4,2020,1150,59800
3,2021,1200,62400
2,2022,1250,65000
0,2023,1300,67600


In [31]:
merged_df['Income_Shift'] = merged_df['Income Per Annum'].shift(1)
merged_df

,Year,Income Per Week,Income Per Annum,Income_Shift
1,2018,1075,55900,NaN
5,2019,1100,57200,55900.0
4,2020,1150,59800,57200.0
3,2021,1200,62400,59800.0
2,2022,1250,65000,62400.0
0,2023,1300,67600,65000.0


In [32]:
merged_df['% Change per Year'] = round((merged_df['Income Per Annum'] /merged_df['Income_Shift']) * 100 - 100,2)
merged_df

,Year,Income Per Week,Income Per Annum,Income_Shift,% Change per Year
1,2018,1075,55900,NaN,NaN
5,2019,1100,57200,55900.0,2.33
4,2020,1150,59800,57200.0,4.55
3,2021,1200,62400,59800.0,4.35
2,2022,1250,65000,62400.0,4.17
0,2023,1300,67600,65000.0,4.00


In [33]:
merged_df = merged_df.drop(columns=['Income_Shift'])
merged_df

,Year,Income Per Week,Income Per Annum,% Change per Year
1,2018,1075,55900,NaN
5,2019,1100,57200,2.33
4,2020,1150,59800,4.55
3,2021,1200,62400,4.35
2,2022,1250,65000,4.17
0,2023,1300,67600,4.00


In [34]:
merged_df.dtypes

Year                   int32
Income Per Week        int32
Income Per Annum       int32
% Change per Year    float64
dtype: object

In [35]:
merged_df.to_csv('income_clean.csv', index = False)